In [1]:
from keras.models import Model
from keras.layers import Input, Activation, merge, Dense, Flatten, concatenate
from keras.layers.convolutional import Convolution2D, MaxPooling2D
from keras.layers import BatchNormalization, add, GlobalAveragePooling2D, AveragePooling2D
from keras.optimizers import Adam
from keras.models import load_model
from keras.datasets import cifar10
from keras.utils import np_utils
import numpy as np


Using TensorFlow backend.


In [0]:
(X_train, y_train), (X_test, y_test) = cifar10.load_data()
X_train = X_train.astype('float32')
X_test = X_test.astype('float32')

X_train /= 255.0
X_test /= 255.0

n_classes = 10
Y_train = np_utils.to_categorical(y_train, n_classes)
Y_test = np_utils.to_categorical(y_test, n_classes)

In [0]:
def InceptionV3_block1():
    def f(x):
        b1 = Convolution2D(64, (1,1), strides=2, padding='same')(x)
        b1 = BatchNormalization()(b1)
        b1 = Activation('relu')(b1)

        b2 = Convolution2D(48, (1,1))(x)
        b2 = BatchNormalization()(b2)
        b2 = Activation('relu')(b2)
        b2 = Convolution2D(96, (3,3), strides=2, padding='same')(b2)
        b2 = BatchNormalization()(b2)
        b2 = Activation('relu')(b2)


        b3 = AveragePooling2D(pool_size=(3, 3), strides=2,  padding='same')(x)
        b3 = Convolution2D(64, (3,3), padding='same')(b3)
        b3 = BatchNormalization()(b3)
        b3 = Activation('relu')(b3)

        b4 = Convolution2D(64, (1,1))(x)
        b4 = BatchNormalization()(b4)
        b4 = Activation('relu')(b4)
        b4 = Convolution2D(96, (3,3), padding='same')(b4)
        b4 = BatchNormalization()(b4)
        b4 = Activation('relu')(b4)
        b4 = Convolution2D(32, (3,3),strides=2, padding='same')(b4)
        b4 = BatchNormalization()(b4)
        b4 = Activation('relu')(b4)

        output = concatenate([b1, b2, b3, b4], axis=-1)
        return output
    return f

In [0]:
def InceptionV3_block2():
    def f(x):
        b1 = Convolution2D(64, (1,1), strides=2,  padding='same')(x)
        b1 = BatchNormalization()(b1)
        b1 = Activation('relu')(b1)

        b2 = Convolution2D(48, (1,1))(x)
        b2 = BatchNormalization()(b2)
        b2 = Activation('relu')(b2)
        b2 = Convolution2D(96, (3,3), strides=2,  padding='same')(b2)
        b2 = BatchNormalization()(b2)
        b2 = Activation('relu')(b2)


        b3 = AveragePooling2D(pool_size=(3, 3), strides=2,  padding='same')(x)
        b3 = Convolution2D(64, (3,3), padding='same')(b3)
        b3 = BatchNormalization()(b3)
        b3 = Activation('relu')(b3)

        b4 = Convolution2D(64, (1,1))(x)
        b4 = BatchNormalization()(b4)
        b4 = Activation('relu')(b4)
        b4 = Convolution2D(96, (3,3), padding='same')(b4)
        b4 = BatchNormalization()(b4)
        b4 = Activation('relu')(b4)
        b4 = Convolution2D(64, (3,3),strides=2, padding='same')(b4)
        b4 = BatchNormalization()(b4)
        b4 = Activation('relu')(b4)

        output = concatenate([b1, b2, b3, b4], axis=-1)
        return output
    return f

In [16]:
def InceptionV3():
    inputs = Input(shape=(32, 32, 3))
    x = Convolution2D(32, (1,1), strides=2)(inputs)
    x = BatchNormalization()(x)
    x = Activation('relu')(x)
    x = MaxPooling2D((3, 3), strides=(2,2), padding='same')(x)
  
    x = InceptionV3_block1()(x)
    x = InceptionV3_block2()(x)
  
    x = GlobalAveragePooling2D()(x)
    x = Dense(10, kernel_initializer='he_normal', activation='softmax')(x)
  
    model = Model(inputs=inputs, outputs=x)
    return model

model = InceptionV3()
adam = Adam()
model.compile(optimizer=adam, loss='categorical_crossentropy', metrics=['accuracy'])
model.summary()
  

__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_5 (InputLayer)            (None, 32, 32, 3)    0                                            
__________________________________________________________________________________________________
conv2d_26 (Conv2D)              (None, 16, 16, 32)   128         input_5[0][0]                    
__________________________________________________________________________________________________
batch_normalization_26 (BatchNo (None, 16, 16, 32)   128         conv2d_26[0][0]                  
__________________________________________________________________________________________________
activation_26 (Activation)      (None, 16, 16, 32)   0           batch_normalization_26[0][0]     
__________________________________________________________________________________________________
max_poolin

In [17]:
batch_size = 50
epochs = 10


h = model.fit(X_train, Y_train,
              batch_size=batch_size,
              epochs=epochs,
              validation_split=0.2,)

Train on 40000 samples, validate on 10000 samples
Epoch 1/10
40000/40000 [==============================] - 38s 956us/step - loss: 1.4259 - acc: 0.4887 - val_loss: 1.2818 - val_acc: 0.5380
Epoch 2/10
40000/40000 [==============================] - 33s 832us/step - loss: 1.1555 - acc: 0.5886 - val_loss: 1.2218 - val_acc: 0.5667
Epoch 3/10
40000/40000 [==============================] - 33s 833us/step - loss: 1.0295 - acc: 0.6366 - val_loss: 1.2112 - val_acc: 0.5718
Epoch 4/10
40000/40000 [==============================] - 33s 835us/step - loss: 0.9211 - acc: 0.6744 - val_loss: 1.1393 - val_acc: 0.5988
Epoch 5/10
40000/40000 [==============================] - 33s 833us/step - loss: 0.8319 - acc: 0.7075 - val_loss: 1.3095 - val_acc: 0.5690
Epoch 6/10
40000/40000 [==============================] - 33s 832us/step - loss: 0.7401 - acc: 0.7400 - val_loss: 1.1308 - val_acc: 0.6166
Epoch 7/10
40000/40000 [==============================] - 33s 830us/step - loss: 0.6493 - acc: 0.7722 - val_loss: 1.